# Simulation of ducts with decreasing openings

In [ ]:
import sys
sys.path.append('..')
import ImpedanceSynthesiser as imps
import numpy as np
import matplotlib.pyplot as pl
%matplotlib notebook

reload(imps)

## Define the ducts

* **world** defines the temperature and humidity
* we build an array of **ducts** with decreasing opening cross-sections

In [ ]:
world = imps.AcousticWorld()

l0 = 1.0
lb = .5

rad0 = .0075
rad_end = [.0075,.0035,.002,.001]

ducts = []

duct = imps.Duct(world=world,losses=True)
duct.set_termination(imps.PerfectOpenEnd())
duct.append_element(imps.StraightDuct(length=lb,radius=rad0))
ducts.append(duct)

for rad_e in rad_end:
    duct = imps.Duct(world=world,losses=True)
    duct.set_termination(imps.PerfectOpenEnd())
    duct.append_element(imps.StraightDuct(length=lb,radius=rad0))
    duct.append_element(imps.StraightDuct(length=l0-lb,radius=rad_e))
    ducts.append(duct)

duct = imps.Duct(world=world,losses=True)
duct.set_termination(imps.PerfectClosedEnd())
duct.append_element(imps.StraightDuct(length=lb,radius=rad0))
ducts.append(duct)

# append an extra 
rext = 0.05
lext = 0.01
#for duct in ducts:
#    duct.append_element(imps.StraightDuct(length=lext,radius=rext))


## Duct impedances, compared

In [ ]:
fvec = np.linspace(10,1000,1000)

#fig,ax=pl.subplots(2,sharex=True)

ax = None
for duct in ducts:
    if ax is None:
        ax=duct.plot_report( fmax=max(fvec), npoints=2000)
    else:
        duct.plot_report(ax=ax, fmax=max(fvec), npoints=2000)
    
ax[1].axhline((np.abs(duct.elements[0].get_characteristic_impedance())),
              lw=1,ls='--',color='r')

f=0
c=duct.speed_of_sound
while f < max(fvec):
    f+=c/(4*l0)
    ax[1].axvline(f,ls='--',lw=1)
    ax[2].axvline(f,ls='--',lw=1)

## Pressure transfer functions

Given two points along the duct (**pos**), what is the ratio of pressures, as a function of frequency? 

In [ ]:
fvec = np.linspace(10,1000,1000)

fig,ax=pl.subplots(2,sharex=True)

pos= [0.005,lb-0.005]

for duct in ducts:
    tf = duct.pressure_transfer_func(fvec,from_pos=pos[0],to_pos=pos[1])
    ax[0].plot(fvec,20*np.log10(np.abs(tf)))
    ax[1].plot(fvec,(np.angle(tf)))
    
ax[0].axhline(0, lw=1, ls='--', color='r')

f=0
c=duct.speed_of_sound
while f < max(fvec):
    f+=c/(4*l0)
    ax[0].axvline(f,ls='--',lw=1)
    ax[1].axvline(f,ls='--',lw=1)

## Element chaining checks

Check that transfer functions are independent of chaining order

In [ ]:
def pressure_transfer_func(duct, freq=1.0, from_pos=0.0,
                           to_pos=None, ref_pos=None, reverse=False):
    """
    get the ratios of pressures at two positions in the duct
    """
    total_length = duct.get_total_length()
    if to_pos is None:
        # sys.stderr.write('\nsetting position to {}\n'.format(total_length))
        end_pos = total_length
    else:
        end_pos = to_pos

    if ref_pos is None:
        ref_pos = total_length

    cmx1 = duct.transfer_mx_at_freq(freq, from_pos=ref_pos,
                                    to_pos=from_pos, reverse=reverse)
    cmx2 = duct.transfer_mx_at_freq(freq, from_pos=ref_pos,
                                    to_pos=to_pos, reverse=reverse)
    z0 = (duct.get_input_impedance_at_freq(freq,
                                           from_pos=ref_pos))

    # set dummy variable to zero if z0 is infinite
    # (this will prevent nan for infinite impedances)
    one = np.isfinite(z0)
    z0[np.logical_not(one)] = 1.
    tfp = (cmx2[0,0]*z0 + cmx2[0,1]*one) / \
          (cmx1[0,0]*z0 + cmx1[0,1]*one)
    return tfp



In [ ]:
fvec = np.linspace(10,1500,1000)

fig,ax=pl.subplots(2,sharex=True)

pos= [0.005,lb-0.005]

duct=ducts[1]

tf = pressure_transfer_func(duct,fvec,from_pos=pos[0],to_pos=pos[1],reverse=False)

ax[0].plot(fvec,20*np.log10(np.abs(tf)))
ax[1].plot(fvec,(np.angle(tf)))

tf = pressure_transfer_func(duct,fvec,from_pos=pos[0],to_pos=pos[1],reverse=True)

ax[0].plot(fvec,20*np.log10(np.abs(tf)))
ax[1].plot(fvec,(np.angle(tf)))

#ax[1].axhline((np.abs(duct.elements[0].get_characteristic_impedance())),
#              lw=1,ls='--',color='r')

f=0
c=duct.speed_of_sound
while f < max(fvec):
    f+=c/(4*l0)
    ax[0].axvline(f,ls='--',lw=1)
    ax[1].axvline(f,ls='--',lw=1)